In [1]:
import numpy as np
import pandas as pd
import gpstk
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', np.RankWarning)
%matplotlib inline

# Get stations baseline < 100 Km

In [2]:
data=pd.read_csv("conus_2003_324_324.csv")
data.columns=["Station", "a","b","c","x","y","z"]

In [3]:
data=data.drop(["a","b","c"],axis=1)
array=data.as_matrix()

In [4]:
def get_pairs(array):
    distances=[]
    for i in range(len(array)):
        for j in range(i+1,len(array)):
            distance=np.sqrt((array[i][1]-array[j][1])**2+(array[i][2]-array[j][2])**2+(array[i][3]-array[j][3])**2)
            if distance<100e3:
                distances.append([array[i][0],array[j][0]])
    return distances

In [5]:
pairs=get_pairs(array)
print "Number of Stations with baseline less than 100 Km ",len(pairs)
pairs[:10]

Number of Stations with baseline less than 100 Km  455


[['1ulm', 'sihs'],
 ['1ulm', 'wnfl'],
 ['abq1', 'nmsf'],
 ['abq1', 'zab1'],
 ['acu1', 'npri'],
 ['adks', 'ang1'],
 ['adks', 'ang2'],
 ['adks', 'lkhu'],
 ['adks', 'netp'],
 ['adks', 'txhu']]

## Functions

In [6]:
def adjust_times(df):
    df=df.sort("TIME")
    times=df.TIME.as_matrix()
    for i in range(times.size):
        if times[i]%30!=0:
            times[i]=times[i]+(30-times[i]%30)
    df["TIME"]=times
    return df

In [7]:
def label_arcs(df):
    time=df.TIME.as_matrix()
    diff=np.diff(time)>3600
    diff=np.hstack((np.array([False]),diff))
    split=np.where(diff==True) #points where there is a "True" value
    
    if len(split[0])>0:
        arcs=np.split(time,split[0])
        arcsID=[]
        n=1
        for i in range(len(arcs)):
            size=len(arcs[i])
            tmp=np.empty(size)
            tmp[:]=n
            arcsID.append(tmp)
            n+=1
        arcsID=np.concatenate(arcsID)
    else:#one arc
        arcsID=np.empty(time.size)
        arcsID[:]=1
        
    df["ARCS"]=arcsID
    return df

In [8]:
def cycle_slips(PhaseDelay,L1,L2,threshold=0.5):
    slips=[]
    slips=np.where(np.abs(np.diff(np.hstack(([0],PhaseDelay))))>threshold) 
    #noL1=np.where(L1==np.nan)[0] #is nan?
    #noL2=np.where(L2==np.nan)[0]
    #print "Slips",slips
    noL1=np.where(np.isnan(L1)==True)[0]#is nan?
    noL2=np.where(np.isnan(L2)==True)[0]
    if len(noL1)>0:
        print "No L1", noL1
    if len(noL2)>0:
        print "No L2", noL2
    
    return slips[0]
    

In [9]:
def subarcs(arc,dfarc,splits):
    time=dfarc.TIME
    if len(splits)>0:
        subarcs=np.split(time,splits)
        subarcsID=[]
        n=arc*10
        for i in range(len(subarcs)):
            size=len(subarcs[i])
            tmp=np.empty(size)
            tmp[:]=n
            subarcsID.append(tmp)
            n+=1
        subarcsID=np.concatenate(subarcsID)
    else:#one arc
        subarcsID=np.empty(time.size)
        subarcsID[:]=arc*10+1
    dfarc["SUBARCS"]=subarcsID
    return dfarc

In [10]:
def del_arcs(df): #delete arcs with less than 10 point
    subarcs=df.SUBARCS.values
    times=[] #Delete this in the other station
    for subarc in np.unique(subarcs):
        if len(np.where(df.SUBARCS==subarc)[0])<10:#.SUBARCS
            #print "Subarc id",subarc," deleted with ",len(np.where(df.SUBARCS==subarc)[0])," datapoints"
            del_points =np.where(df.SUBARCS==subarc)[0] #Save times so we can delete in other station
            times.append(df.TIME[del_points].values)
            df=df[df.SUBARCS!=subarc]
            
    if len(times)>0:
        times=np.concatenate(times)
        times=times[np.isnan(times)==False]
        times=np.unique(times)
    else:
        times=None
    
    return df,times
            

In [11]:
def poly_fit(dfarc):
    #receives a dataframe with a number of subarcs
    #On each subarc takes N elements from LI=L1-L2 and performs interpolation, 
    
    f1,f2=gpstk.L1_FREQ_GPS,gpstk.L2_FREQ_GPS
    alfa=1.0/((f1**2/f2**2)-1)
    #lists potential outliers and time when it occurs on each subarc
    times,gradients=[],[]
    subarcs=np.unique(dfarc.SUBARCS.values)
    for subarc in subarcs:
        dfsubarc=dfarc[dfarc.SUBARCS==subarc]
        N=10 #window 
        tPoly ,Poly=[],[]
        lI=alfa*(dfsubarc.L1.as_matrix()-dfsubarc.L2.as_matrix())
        time=dfsubarc.TIME.as_matrix()

        for i in range(0,lI.size,N): 
            x=np.array(time[i:i+N])
            y=np.array(lI[i:i+N])
            z= np.polyfit(x,y,2)
            p = np.poly1d(z)
            for i in range(x.size):
                Poly.append(p(x[i]))
                tPoly.append(x[i]) 
                
        Poly=np.array(Poly)
        
        #detects datajumps in the diference between the polinomyal fit and real data 
        residual=lI-Poly
        candidates=np.where(np.abs(np.diff(np.hstack(([0],residual))))>0.8)[0]

        if len(candidates)>0:
            time_candidates=time[candidates]
            gradient_candidates=residual[candidates]
            times.append(time_candidates) #array candidatos en subarcos
            gradients.append(gradient_candidates)
        else:
            times.append(np.array([]))
            gradients.append(np.array([]))
    
        dfarc.POLYFIT[dfarc.SUBARCS==subarc]=Poly

    return dfarc,times,gradients #df con columna de interpolacion,  array de arrays(algunos vacios), array dde arrays o None (uno por cada sub arco)

In [12]:
def outlier_detect(dfarc,k=10):#k=30? 15 min
    f1,f2=gpstk.L1_FREQ_GPS,gpstk.L2_FREQ_GPS
    alfa=1.0/((f1**2/f2**2)-1)
    #lists outlier factors on each arc
    outliers_subarcs,times_subarcs=[],[]
    subarcs=np.unique(dfarc.SUBARCS.values)
    for subarc in subarcs:
        dfsubarc=dfarc[dfarc.SUBARCS==subarc]
        lI=alfa*(dfsubarc.L1.as_matrix()-dfsubarc.L2.as_matrix())
        times=dfsubarc.TIME.as_matrix()
        times_subarcs.append(times)
        outliers=[] #set of outlier factors for every element in L=L1-L2
    
        for i in range(0,lI.size):
            if i<(k/2+1):
                neighbours=np.hstack((lI[0:i],lI[i+1:i+(k/2)+1])) #neighbours around i, without i
                tn=np.hstack((times[0:i],times[i+1:i+(k/2)+1]))

            elif i>lI.size-(k/2+1):
                neighbours=np.hstack((lI[i-k/2:i],lI[i+1:lI.size+1]))
                tn=np.hstack((times[i-k/2:i],times[i+1:lI.size+1])) #times neighbour

            else:
                neighbours=np.hstack((lI[i-k/2:i],lI[i+1:i+(k/2)+1]))
                tn=np.hstack((times[i-k/2:i],times[i+1:i+(k/2)+1]))

            OFt=0
            deno=np.sum(1/(np.abs(times[i]-tn)*1.0))#denominator de Wpq para elemento i
            for neighbour in range(neighbours.size): 
                if times[neighbour]!=times[i]:
                    Wpq=1/np.abs(times[i]-times[neighbour])
                    Wpq=Wpq/deno
                    OFt+=(Wpq*np.abs(lI[i]-lI[neighbour]))
            outliers.append(OFt)  #outliers in the subarc
        outliers_subarcs.append(outliers) #outliers on arc 

    return outliers_subarcs,times_subarcs #list of lists(suarcs),list of arrays


In [23]:
def remove_outliers(ptimes,otimes,poly_outliers,outliers_factor,subarcs):
    confirmed=[] #times of confirmed outliers
    print "# Subarcs: ",subarcs
    print "Polinomial",poly_outliers
    for subarc in range(subarcs):
        if len(poly_outliers[subarc])>0:
            for i in range(len(poly_outliers[subarc])):
                print "Quedan", len(poly_outliers[subarc])," candidatos" 
                #check for max outliers with polinomial method
                maxpoly=np.argmax(poly_outliers[subarc])
                valuemaxpoly=poly_outliers[subarc][maxpoly] 
                tmaxpoly=ptimes[subarc][maxpoly]
                #check for max outliers with outliers factor method
                maxofactor=np.argmax(outliers_factor[subarc])
                valuemaxout=outliers_factor[subarc][maxofactor]
                tmaxofactor=otimes[subarc][maxofactor]

                if tmaxofactor==tmaxpoly:
                    if tmaxofactor not in confirmed:
                        confirmed.append(tmaxofactor)
                        print "Outlier detectado en t=",tmaxofactor
                        #Save time of confirmed outlier
                        print "Verificar que elemento se elimina"
                        print len(outliers_factor[subarc])
                        print len(otimes[subarc])
                        print poly_outliers[subarc].shape
                        print ptimes[subarc].shape
                        #Delete point from both methods and continue 
                        outliers_factor[subarc]=np.delete(outliers_factor[subarc],maxofactor)
                        otimes[subarc]=np.delete(otimes[subarc],maxofactor)
                        poly_outliers[subarc]=np.delete(poly_outliers[subarc],maxpoly)
                        ptimes[subarc]=np.delete(ptimes[subarc],maxpoly)
                        print len(outliers_factor[subarc])
                        print len(otimes[subarc])
                        print poly_outliers[subarc].shape
                        print ptimes[subarc].shape
                    else:
                        print "Ya estaba confirmado"
                        break
                        

                else: #If there aret any more coincidences stop searching
                    print "No hay mas confirmados"
                    break

        else:
            #See next subarc
            print "No hay candidatos con metodo polinomial fit"
            continue
            
    return confirmed
            

    
    

# Preprocessing of ten Pairs of stations


In [28]:
from os import listdir
dir_txt="txtcors/"
files = [f for f in listdir(dir_txt)]

f1=gpstk.L1_FREQ_GPS
f2=gpstk.L2_FREQ_GPS
factor_alfa=f2**2/(f1**2-f2**2)
c=3e8
alfa=1.0/((f1**2/f2**2)-1) 

for stations in pairs[:1]:
    #Load Files
    st1,st2=stations[0],stations[1]
    columns=["PRN","TIME","C1","C2","L1","L2","Tgd","IPP","Elevation","Azimuth"]
    file1=dir_txt+[f for f in files if st1 in f ][0]
    file2=dir_txt+[f for f in files if st2 in f ][0]
    
    df1=pd.read_csv(file1,sep=",")
    df1.columns=columns
    df2=pd.read_csv(file2,sep=",")
    df2.columns=columns
    df1,df2=adjust_times(df1),adjust_times(df2)
    df3=pd.merge(df1,df2,on=["TIME","PRN","Tgd"])
    
    #For each satellite observed by the stations
    for sat in np.unique(df3.PRN.values):
    ##
        df=df3[df3.PRN==sat] #dataframe with times of an specific satellite
        df=df.reset_index(drop=True)
        #Estimate delay measures 
        df["PhaseDelay_x"]=alfa*(df.L1_x-df.L2_x)
        df["PhaseDelay_y"]=alfa*(df.L1_y-df.L2_y)
        df["CodeDelay_x"]=alfa*(df.C2_x-df.C1_x)
        df["CodeDelay_y"]=alfa*(df.C2_y-df.C1_y)
        #Add column with indicators of time separation
        df=label_arcs(df)
       
        for arc in np.unique(df.ARCS.values):
            #Search for cycle Slips on each arc
            dfarc=df[df.ARCS==arc]
            slips1=cycle_slips(dfarc.PhaseDelay_x.as_matrix(),dfarc.L1_x.as_matrix(),dfarc.L2_x.as_matrix(),2.5)
            slips2=cycle_slips(dfarc.PhaseDelay_y.as_matrix(),dfarc.L1_y.as_matrix(),dfarc.L2_y.as_matrix(),2.5)
            #Dataframes with subarcs on stations
            dfarc1=subarcs(arc,dfarc,slips1)
            dfarc1=dfarc1.drop(["PhaseDelay_y","CodeDelay_y","L1_y","L2_y","C1_y","C2_y","IPP_y","Elevation_y","Azimuth_y"],axis=1)
            dfarc2=subarcs(arc,dfarc,slips2)
            dfarc2=dfarc2.drop(["PhaseDelay_x","CodeDelay_x","L1_x","L2_x","C1_x","C2_x","IPP_x","Elevation_x","Azimuth_x"],axis=1)
            #Remove short-arcs 
            new_dfarc1,times1=del_arcs(dfarc1)
            new_dfarc2,times2=del_arcs(dfarc2)
            if times1!=None:
                for t in times1:
                    new_dfarc2=new_dfarc2[new_dfarc2.TIME!=t]
            if times2!=None:
                for t in times2:
                    new_dfarc1=new_dfarc1[new_dfarc1.TIME!=t]
            
            columns=['PRN','TIME','C1','C2','L1','L2','Tgd', 'IPP', 'Elevation', 'Azimuth', 'PhaseDelay', 'CodeDelay', 'ARCS', 'SUBARCS']
            new_dfarc1.columns=columns
            new_dfarc2.columns=columns
            #Polinomial fit and outlier detection
            new_dfarc1["POLYFIT"]=np.nan
            new_dfarc2["POLYFIT"]=np.nan
            new_dfarc1,ptimes1,poly_outliers1=poly_fit(new_dfarc1)
            new_dfarc2,ptimes2,poly_outliers2=poly_fit(new_dfarc2)
            #Outliers factor
            outliers1,otimes1=outlier_detect(new_dfarc1,10)
            outliers2,otimes2=outlier_detect(new_dfarc2,10)
            #if len(ptimes1) != len(otimes1):
                #print len(ptimes1), len(otimes1)
            #Confirm and delete outliers
            #station 1
            subarcs1=len(np.unique(new_dfarc1.SUBARCS.values))
            times1=remove_outliers(ptimes1,otimes1,poly_outliers1,outliers1,subarcs1)
            #station 2
            subarcs2=len(np.unique(new_dfarc2.SUBARCS.values))
            times2=remove_outliers(ptimes2,otimes2,poly_outliers2,outliers2,subarcs2)
            #Both dataframes now should have same number of observations and we can merge them
            
            #print np.unique(dfarc2.SUBARCS.values)
        break

# Subarcs:  1
Polinomial [array([], dtype=float64)]
No hay candidatos con metodo polinomial fit
# Subarcs:  1
Polinomial [array([ 0.87293814, -0.92452953,  0.83139617, -0.71471635, -1.46130593,
        4.3490522 , -1.87615798, -0.30972705,  0.79456928, -0.26971112,
        1.62879513, -1.79988355,  0.32942796, -1.18830028,  3.06371093,
       -1.53096966,  1.44815091, -1.26991439,  3.4038376 , -1.86580963,
       -0.3062582 ,  0.79449206, -1.78302233, -0.12158365,  0.95128535,
       -0.05409582,  0.8808661 , -1.65945032,  2.55146826, -3.19703094,
       -0.03408492,  1.09657683,  2.74472196, -2.91040302,  0.58143204,
        1.09977536, -0.697014  ,  0.94505329, -0.70215084,  0.10707826,
        1.21736232, -0.87333042, -1.33206905, -0.29267605,  1.31646273,
       -0.98998202,  0.95388479, -0.18501476,  1.07559595, -0.81453449,
       -2.92294405,  3.99524147, -0.28358465,  0.96312851, -0.57989579,
        1.22334021, -0.90285772, -0.04815897,  1.47853331, -1.37061275,
        1.1223

In [ ]:
print "*******************************************************************************"
print "****************************POLYNOMIAL FIT*************************************"
print "Candidates Times st1: ",ptimes1,"\n"
print "Outliers: ",poly_outliers1
print "Candidates Times st2: ",ptimes2,"\n"
print "Outliers: ",poly_outliers2
#Outliers removal
print "****************************OUTLIER FACTOR METHOD*************************************"
print "Candidates Times st1: ",otimes1,"\n"
print "Outliers: ",outliers1
print "Candidates Times st2: ",otimes2,"\n"
print "Outliers: ",outliers2

In [ ]:
print "Subarcs",np.unique(new_dfarc1.SUBARCS.values)
print "Subarcs",np.unique(new_dfarc2.SUBARCS.values)
print 
#new_dfarc1.PhaseDelay

In [ ]:
print len(ptimes1[0])
print len(otimes1[0])

In [ ]:
print len(np.unique(new_dfarc1.SUBARCS.values))
np.unique(new_dfarc1.SUBARCS.values)

In [ ]:
subarc=0
print len(poly_outliers1[subarc])>0
print poly_outliers1
print outliers1

In [ ]:
print len(ptimes2) #un solo subarco
print ptimes2[0].size
print poly_outliers2[0].size

In [ ]:
print len(otimes1[0]) #un solo sub arco
print len(outliers1[0])

In [ ]:
plt.scatter(new_dfarc1.TIME[new_dfarc1.SUBARCS==17.],new_dfarc1.PhaseDelay[new_dfarc1.SUBARCS==17.])

In [ ]:
plt.scatter(new_dfarc1.TIME[new_dfarc1.SUBARCS==17.],new_dfarc1.POLYFIT[new_dfarc1.SUBARCS==17.])
print poly_outliers1
print poly_outliers2


In [ ]:
new_dfarc1[new_dfarc1.SUBARC==17.0]

In [ ]:
new_dfarc2.TIME

In [ ]:
#plt.scatter(dfarc1.TIME,dfarc1.SUBARCS)
#print np.unique(dfarc1.SUBARCS.values)
#dfarc1[dfarc1.SUBARCS==11].SUBARCS
#Compare before and after remove short arcs
#plt.ylim(-1000,100000)
print np.unique(dfarc1.SUBARCS.values)
plt.scatter(dfarc1.TIME,dfarc1.PhaseDelay_x,color="r",alpha=.5)
#for subarc in 
#plt.scatter(new_dfarc.TIME,new_dfarc1.PhaseDelay_x,color="b",alpha=.5)
#again merge by time, this removes short arcs y both stations
#print np.unique(new_dfarc1.SUBARCS.values)
#print np.unique(new_dfarc1.SUBARCS.values)